In [1]:
import json
import os

from oddstracker.config import DATA_DIR
from oddstracker.service.oddscollector import pull_kdata

SQL tables already exist: ['betoffer', 'event']


In [ ]:
kdata = pull_kdata()

In [ ]:
with open(os.path.join(DATA_DIR, "kdata.json"), "w", encoding="utf-8") as f:
    _test_data = kdata[1]
    out = ""
    for bo in _test_data.betOffers:
        out += bo.model_dump_json() + "\n"
    f.write(out)

In [ ]:
with open(os.path.join(DATA_DIR, "betOffers.json"), "w") as f:
    json.dump(kdata[0].betOffers[0].model_dump(), f, indent=2, default=str)


In [ ]:
import nflreadpy as nfl

nfl.load_teams()

season,team,nfl,nfl_team_id,espn,pfr,pff,pfflabel,fo,full,location,short_location,nickname,hyphenated,sbr,sbr_wins,sbr_name,draft_kings
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,i64,str,str,str
2002,"""ARI""","""ARI""",3800,"""ARI""","""CRD""",null,null,"""ARI""","""Arizona Cardinals""","""Arizona""","""Arizona""","""Cardinals""","""arizona-cardinals""",1549,null,null,null
2002,"""ATL""","""ATL""",200,"""ATL""","""ATL""",null,null,"""ATL""","""Atlanta Falcons""","""Atlanta""","""Atlanta""","""Falcons""","""atlanta-falcons""",1546,null,null,null
2002,"""BAL""","""BAL""",325,"""BAL""","""RAV""",null,null,"""BAL""","""Baltimore Ravens""","""Baltimore""","""Baltimore""","""Ravens""","""baltimore-ravens""",1521,null,null,null
2002,"""BUF""","""BUF""",610,"""BUF""","""BUF""",null,null,"""BUF""","""Buffalo Bills""","""Buffalo""","""Buffalo""","""Bills""","""buffalo-bills""",1526,null,null,null
2002,"""CAR""","""CAR""",750,"""CAR""","""CAR""",null,null,"""CAR""","""Carolina Panthers""","""Carolina""","""Carolina""","""Panthers""","""carolina-panthers""",1545,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025,"""SEA""","""SEA""",4600,"""SEA""","""SEA""","""29""","""SEA""","""SEA""","""Seattle Seahawks""","""Seattle""","""Seattle""","""Seahawks""","""seattle-seahawks""",1548,null,"""Seattle Seahawks""","""SEA Seahawks"""
2025,"""SF""","""SF""",4500,"""SF""","""SFO""","""28""","""SF""","""SF""","""San Francisco 49ers""","""San Francisco""","""San Francisco""","""49ers""","""san-francisco-49ers""",1547,null,"""San Francisco 49ers""","""SF 49ers"""
2025,"""TB""","""TB""",4900,"""TB""","""TAM""","""30""","""TB""","""TB""","""Tampa Bay Buccaneers""","""Tampa Bay""","""Tampa Bay""","""Buccaneers""","""tampa-bay-buccaneers""",1544,null,"""Tampa Bay Buccaneers""","""TB Buccaneers"""


In [ ]:
import nflreadpy as nfl

# nfl.import_weekly_pfr('pass',[2025])
nfl.load_teams()

season,team,nfl,nfl_team_id,espn,pfr,pff,pfflabel,fo,full,location,short_location,nickname,hyphenated,sbr,sbr_wins,sbr_name,draft_kings
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,i64,str,str,str
2002,"""ARI""","""ARI""",3800,"""ARI""","""CRD""",null,null,"""ARI""","""Arizona Cardinals""","""Arizona""","""Arizona""","""Cardinals""","""arizona-cardinals""",1549,null,null,null
2002,"""ATL""","""ATL""",200,"""ATL""","""ATL""",null,null,"""ATL""","""Atlanta Falcons""","""Atlanta""","""Atlanta""","""Falcons""","""atlanta-falcons""",1546,null,null,null
2002,"""BAL""","""BAL""",325,"""BAL""","""RAV""",null,null,"""BAL""","""Baltimore Ravens""","""Baltimore""","""Baltimore""","""Ravens""","""baltimore-ravens""",1521,null,null,null
2002,"""BUF""","""BUF""",610,"""BUF""","""BUF""",null,null,"""BUF""","""Buffalo Bills""","""Buffalo""","""Buffalo""","""Bills""","""buffalo-bills""",1526,null,null,null
2002,"""CAR""","""CAR""",750,"""CAR""","""CAR""",null,null,"""CAR""","""Carolina Panthers""","""Carolina""","""Carolina""","""Panthers""","""carolina-panthers""",1545,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025,"""SEA""","""SEA""",4600,"""SEA""","""SEA""","""29""","""SEA""","""SEA""","""Seattle Seahawks""","""Seattle""","""Seattle""","""Seahawks""","""seattle-seahawks""",1548,null,"""Seattle Seahawks""","""SEA Seahawks"""
2025,"""SF""","""SF""",4500,"""SF""","""SFO""","""28""","""SF""","""SF""","""San Francisco 49ers""","""San Francisco""","""San Francisco""","""49ers""","""san-francisco-49ers""",1547,null,"""San Francisco 49ers""","""SF 49ers"""
2025,"""TB""","""TB""",4900,"""TB""","""TAM""","""30""","""TB""","""TB""","""Tampa Bay Buccaneers""","""Tampa Bay""","""Tampa Bay""","""Buccaneers""","""tampa-bay-buccaneers""",1544,null,"""Tampa Bay Buccaneers""","""TB Buccaneers"""
